In [1]:
import tensorflow as tf
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import time
import random

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import ops

from pandas_ml import ConfusionMatrix

## load all npz files from npz_files directory.

In [2]:
import os

npz_files_directory_train = '/home/rohan/Desktop/label-generator/raw_files/npz-files/train'
npz_files_directory_test = '/home/rohan/Desktop/label-generator/raw_files/npz-files/test'
# collect all files from npz directory.
train_files = list()
for f in os.listdir(npz_files_directory_train):
    train_files.append(f)
    
test_files = list()
for f in os.listdir(npz_files_directory_test):
    test_files.append(f)

In [3]:
train_files = [file for file in train_files if file[-3:] == 'npz']
test_files = [file for file in test_files if file[-3:] == 'npz']

In [4]:
print('number of train files', len(train_files))
print('number of test files', len(test_files))

number of train files 5
number of test files 3


## load all numpy arrays into training data x and y.
structure of the npz files = ['c1', 'c2', 'c3', 'Z', 'labels'].
c1, c2, c3 => numpy array containing all class 1, 2, 3 bounding boxes respectively.
Z => the raw PSD files.
labels => pixel wise labels for psd files. same dimensions as the PSD files.

In [5]:
def load_data(directory, files_list):
    training_data_x = list()
    training_data_y = list()

    count = 0
    for file_name in files_list:
        data1 = np.load(directory + '/' + file_name)
        training_data_x.append(data1['Z'])
        training_data_y.append(data1['labels'])
        count += 1
    print('files loaded', count)
    return training_data_x, training_data_y 

In [6]:
training_data_x, training_data_y = load_data(npz_files_directory_train, train_files)

files loaded 5


In [7]:
test_data_x, test_data_y = load_data(npz_files_directory_test, test_files)

files loaded 3


In [8]:
print('shape of training_data_x', len(training_data_x))
print('shape of training_data_y', len(training_data_y))

shape of training_data_x 5
shape of training_data_y 5


In [9]:
print('shape of training_data_x', len(test_data_x))
print('shape of training_data_y', len(test_data_y))

shape of training_data_x 3
shape of training_data_y 3


In [10]:
print('shape of training_data_x samples', training_data_x[0].shape)
print('shape of training_data_y samples', training_data_y[0].shape)

shape of training_data_x samples (1953, 512)
shape of training_data_y samples (1953, 512)


## variable structures:-
1. train_x contains all Z for all files.
2. train_y contains all label for all files.

## convert training_data_x to shape (size of all timesteps, 512)

In [11]:
def processFrequencies(freq, num_steps):
    frequencies = np.zeros((512, num_steps), dtype=np.float32)
    assert freq.shape[0] == 512
    freq = np.reshape(freq, (512, 1))
    for i in range(freq.shape[0]):
        if freq[i:i+num_steps, 0].shape[0] == num_steps:
            frequencies[i, :] = freq[i:i+num_steps, 0]
        else:
#             print(freq[i:i+num_steps, 0].shape[0])
            frequencies[i, :] = np.pad(freq[i:i+num_steps, 0], (0, num_steps-freq[i:i+num_steps, 0].shape[0]), 'edge')
    frequencies = np.reshape(frequencies, (512, num_steps, 1))
    return frequencies

In [12]:
def processLabels(labels):
    labels = np.reshape(labels, (512, 1))
    labels_reshaped = np.zeros((512, 4))
    for i in range(labels.shape[0]):
        labels_reshaped[i, labels[i, 0]] = 1
    return labels_reshaped

## Convert all x_train data to serial x_train data.

In [13]:
# global parameters.
x_train, y_train, x_test, y_test = list(), list(), list(), list()
num_steps = 8

In [14]:
def convertToSerialList(x, y, x_copy, y_copy):
    assert len(x) == len(y)
    for i in range(len(x)):
        for j in range(x[i].shape[0]):
            x_copy.append(processFrequencies(x[i][j,:] , num_steps))
            y_copy.append(processLabels(y[i][j,:]))

In [15]:
convertToSerialList(training_data_x, training_data_y, x_train, y_train)

In [16]:
print('length of x is ', len(x_train))
print('length of y is ', len(y_train))

length of x is  9765
length of y is  9765


In [17]:
print('shape of x is ', x_train[0].shape)
print('shape of x is ', y_train[0].shape)

shape of x is  (512, 8, 1)
shape of x is  (512, 4)


In [18]:
convertToSerialList(test_data_x, test_data_y, x_test, y_test)

In [19]:
print('length of x is ', len(x_test))
print('length of y is ', len(y_test))

length of x is  5858
length of y is  5858


In [20]:
print('shape of x is ', x_test[0].shape)
print('shape of x is ', y_test[0].shape)

shape of x is  (512, 8, 1)
shape of x is  (512, 4)


In [21]:
# deleting unused files.
del training_data_x, training_data_y, test_data_x, test_data_y

## Creating the RNN model.

In [22]:
# RNN Global Parameter models.
num_steps = 8
frequency_shape = [512, num_steps, 1]
labels_shape = [512, 4]

hidden_layer_dimension = 32
number_layers = 2
use_dropout = False
dropout = 0.0
num_layers = 2
batch_size = 128
learning_rate = 0.001
num_epochs = 200
step_size = 512 // batch_size
num_samples_train = len(x_train)
num_samples_test = len(x_test)
shuffled_indexed_train = [i for i in range(num_samples_train)]
shuffled_indexed_test = [i for i in range(num_samples_test)]
prediction_list = list()

In [23]:
class Input:
    def __init__(self, frequencies, labels, is_train=True):
        """
        creates two run able objects -> inputs for feeding inputs and labels.
        """
        self.is_train = is_train
        self.frequencies = None
        self.labels = None
        self.getData()
#         self.dataset = tf.data.Dataset.from_tensor_slices((self.frequencies, self.labels))
#         if is_train:
#             self.dataset = self.dataset.shuffle(buffer_size=num_samples_train)
#         self.dataset = self.dataset.apply(tf.contrib.data.unbatch())
#         self.dataset = self.dataset.batch(batch_size)
#         self.dataset = self.dataset.prefetch(512 // batch_size)
        
#         self.iterator = self.dataset.make_initializable_iterator()
        
    def getOutputType(self):
        return self.dataset.output_types
    
    def getOutputShape(self):
        return self.dataset.output_shapes
    
    def getData(self):
        if is_train:
            training_x, training_y = load_data(npz_files_directory_train, train_files)
            print('shape of training_data_x', len(training_x))
            print('shape of training_data_y', len(training_y))
            print('shape of training_data_x samples', training_x[0].shape)
            print('shape of training_data_y samples', training_y[0].shape)
            train_x, train_y = list(), list()
            convertToSerialList(training_x, training_y, train_x, train_y)
            self.frequencies = np.array(train_x)
            self.labels = np.array(train_y)
        else:
            testing_x, testing_y = load_data(npz_files_directory_test, test_files)
            test_x, test_y = list(), list()
            convertToSerialList(testing_x, testing_y, test_x, test_y)
            self.frequencies = np.array(test_x)
            self.labels = np.array(test_y)

In [24]:
class Model:
    def __init__(self, inputs, output_labels):
        self.init_state = tf.placeholder(tf.float32, [number_layers, 2, batch_size, hidden_layer_dimension])
        state_per_layer_list = tf.unstack(self.init_state, axis=0)
        rnn_state_tuples = tuple([tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]) for state in state_per_layer_list])

        if use_dropout:
            cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)
            
        self.state = current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))

        if num_layers > 1:
            cell_list = [self.createLSTMCells() for _ in range(num_layers)]
            cell = tf.nn.rnn_cell.MultiRNNCell(cell_list, state_is_tuple=True)
        elif num_layers == 1:
            cell = createLSTMCells()

        self.output, self.state = tf.nn.dynamic_rnn(cell, 
                                          inputs,
                                          dtype=tf.float32, 
                                          initial_state=rnn_state_tuples)
        
        # extract the last output for time=num_seq from the output.
        self.output = tf.transpose(self.output, [1, 0, 2])
        self.output = tf.gather(self.output, int(self.output.shape[0]-1))
        
        # softmax layer
        dense_layer = tf.layers.dense(self.output, units=4, activation=tf.nn.relu)
        self.logits = tf.layers.dense(dense_layer, units=4, activation=None)
        
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=output_labels,
                                                                        logits=self.logits)
        self.loss = tf.reduce_mean(self.cross_entropy)
        # optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        
        # measuring accuracy.
        prediction = tf.nn.softmax(self.logits)
        self.prediction_class = tf.argmax(prediction, axis=1)
        output_labels_class = tf.argmax(output_labels, axis=1)
        matching_prediction = tf.equal(output_labels_class, self.prediction_class)
        self.accuracy = tf.reduce_mean(tf.cast(matching_prediction, tf.float32))
        self.accuracy_dict = dict()
        
    def plotAccuracy(self):
        lists = sorted(self.accuracy_dict.items()) # sorted by key, return a list of tuples
        x, y = zip(*lists) # unpack a list of pairs into two tuples
        plt.plot(x, y)
        plt.show()
        
    def saveModel(self, epoch):
        output_node_names = "accuracy, prediction, state"
        output_graph_definition = graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        )
        model_save_path = ('./saved_model/model_'+str(epoch)+'.pb')
        with tf.gfile.GFile(model_save_path, "wb") as f:
            f.write(output_graph_definition.SerializeToString())
        
    def createLSTMCells(self):
        cell = tf.contrib.rnn.LSTMCell(hidden_layer_dimension, forget_bias=1.0)
        return cell
    
    def calculateTestAccuracy(self, epoch):
        sess.run(test_dataset.iterator.initializer)
        
        test_accuracy_list = list()
        label_list = list()
        current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))
        counter = 0
        # looping till all values are consumed
        while True:
            try:
                accuracy, prediction_class, current_state, _, _ = sess.run([m.accuracy, m.prediction_class, m.state, 
                                                                      input_data, label_data], 
                                                    feed_dict={m.init_state: current_state, handle: testing_handle})
                test_accuracy_list.append(accuracy)
                label_list.append(prediction_class)
                counter += 1
                if counter != 0 and counter % step_size == 0:
                    current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))
            except tf.errors.OutOfRangeError:
                break
        print('test accuracy at epoch #', epoch, 'is ', (sum(test_accuracy_list) / len(test_accuracy_list)))
        self.accuracy_dict[epoch] = (sum(test_accuracy_list) / len(test_accuracy_list) )
        print(len(label_list))
        return label_list
    
    def printConfusionMatrix(self, prediction):
        prediction_np = np.array(prediction)
        pred_np = np.zeros((num_samples_test, 512))
        row, col = 0, 0
        for i in range(prediction_np.shape[0]):
            pred_np[row, col:col+batch_size] = prediction_np[i]
            col += batch_size
            if col == 512:
                col = 0
                row += 1
        
        y_test_np = np.argmax(y_test, axis=2)
        y_test_np = np.reshape(y_test_np, [y_test_np.shape[0] * y_test_np.shape[1]])
        pred_np = np.reshape(pred_np, [pred_np.shape[0] * pred_np.shape[1]])
        cm = ConfusionMatrix(y_test_np, pred_np)
        print(cm)

In [ ]:
cell_start_time = time.time()

# creating training and testing dataset.
train_dataset = Input(np.array(x_train), np.array(y_train))
test_dataset = Input(np.array(x_test), np.array(y_test), is_train=False)
handle = tf.placeholder(tf.string, [])
iterator = tf.data.Iterator.from_string_handle(handle, 
                                               train_dataset.dataset.output_types,
                                               train_dataset.dataset.output_shapes)
input_data, label_data = iterator.get_next()

# test_input = Input(is_test=True)
m = Model(input_data, label_data)
prediction = None
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    counter = 0
    
    training_handle = sess.run(train_dataset.iterator.string_handle())
#     print(training_handle)
#     print(training_handle.shape)
    testing_handle = sess.run(test_dataset.iterator.string_handle())
    
    for epoch in range(num_epochs):
        print('epoch #', epoch, 'started')
        
        # variables.
        epoch_time = time.time()
        counter = 0
        current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))
        train_accuracy_list = list()
        
        # loading the dataset
        sess.run(train_dataset.iterator.initializer)
        
        # looping till all values are consumed
        while True:
            try:
                if epoch % 10 == 0:
                    parameter1, accuracy, current_state, _, _ = sess.run([m.optimizer, m.accuracy, m.state, 
                                                                    input_data, label_data], 
                                                                   feed_dict={m.init_state: current_state, 
                                                                              handle: training_handle})
                    train_accuracy_list.append(accuracy)
                    counter += 1
                    if counter != 0 and counter % step_size == 0:
                        current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))   
                else:
                    parameter1, current_state, _, _ = sess.run([m.optimizer, m.state, input_data, label_data], 
                                                             feed_dict={m.init_state: current_state, 
                                                                        handle: training_handle})
                    counter += 1
                    if counter != 0 and counter % step_size == 0:
                        current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))
            except tf.errors.OutOfRangeError:
                break
            
        # after training, if epoch number is multiple of 10.
        if epoch % 10 == 0:
            print('train set accuracy at epoch #', epoch, 'is ', (sum(train_accuracy_list) / len(train_accuracy_list)))
            saver.save(sess, './checkpoints/model_checkpoint_'+str(epoch))
            prediction = m.calculateTestAccuracy(epoch)
            m.printConfusionMatrix(prediction)
        end_time = time.time()
        print('epoch #', epoch, 'ended - ',  end_time - epoch_time)
    saver.save(sess, './checkpoints/model_checkpoint_final')
    prediction = m.calculateTestAccuracy(epoch)
    m.printConfusionMatrix(prediction)
#     m.saveModel(epoch)
cell_end_time = time.time()
print('total time to train the network is ', cell_end_time - cell_start_time)

b'\n,/job:localhost/replica:0/task:0/device:CPU:0\x12\tlocalhost\x1a\x0b_0_Iterator \xa4\xd6\xcb\xd1\xa6\xeb\xb2\xc1\xc7\x01*.N10tensorflow12_GLOBAL__N_116IteratorResourceE'
epoch # 0 started
train set accuracy at epoch # 0 is  0.9078028993855607
test accuracy at epoch # 0 is  0.9167117883663366
23432


/home/rohan/anaconda3/envs/tensorflow-env/lib/python3.6/site-packages/pandas_ml/confusion_matrix/abstract.py:66: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  df = df.loc[idx, idx.copy()].fillna(0)  # if some columns or rows are missing


Predicted      0.0  1.0  2.0  3.0  __all__
Actual                                    
0.0        2749490    0    0    0  2749490
1.0         153052    0    0    0   153052
2.0          76629    0    0    0    76629
3.0          20125    0    0    0    20125
__all__    2999296    0    0    0  2999296
epoch # 0 ended -  312.3646020889282
epoch # 1 started
epoch # 1 ended -  217.60556530952454
epoch # 2 started
epoch # 2 ended -  217.40476965904236
epoch # 3 started
epoch # 3 ended -  218.78481578826904
epoch # 4 started
epoch # 4 ended -  212.63600826263428
epoch # 5 started
epoch # 5 ended -  212.21049642562866
epoch # 6 started
epoch # 6 ended -  210.87747597694397
epoch # 7 started
epoch # 7 ended -  211.63655805587769
epoch # 8 started
epoch # 8 ended -  209.2229540348053
epoch # 9 started
epoch # 9 ended -  212.73419976234436
epoch # 10 started
train set accuracy at epoch # 10 is  0.9078028993855607
test accuracy at epoch # 10 is  0.9167117883663366
23432
Predicted      0.0  1.0  2.

In [ ]:
m.printConfusionMatrix(prediction)

In [ ]:
m.plotAccuracy()

Predicted      0.0     1.0    2.0    3.0  __all__
Actual                                           
0.0        2591457  122997  23270  11766  2749490
1.0         103267   45613   2025   2147   153052
2.0          60231    4473   9225   2700    76629
3.0          14268    2793   1423   1641    20125
__all__    2769223  175876  35943  18254  2999296